## Import Necessary Libraries

In [86]:
import pandas as pd
import numpy as np
import uuid
import json
from pathlib import Path

## WORKING IN NEW STRATEGY...

In [87]:
SHORT_WINDOW = 50
LONG_WINDOW = 200

MOMENTUM_WINDOW = 20
RSI_WINDOW = 14
VOLATILITY_THRESHOLD = 3

class DefaultStrategy:
    def __init__(self):
        self.initialized = False
        
        # Price history for each pair - this maintains state between calls
        self.price_history = {
            "token_1/fiat": [],
            "token_2/fiat": [],
            "token_1/token_2": []
        }
        
        self.short_window = SHORT_WINDOW
        self.long_window = LONG_WINDOW

        self.momentum_window = MOMENTUM_WINDOW
        self.rsi_window = RSI_WINDOW
        # Volatility threshold for signals
        self.threshold = VOLATILITY_THRESHOLD

        self.initial_balance = 0
        self.previous_balance = 0
        self.profit = 0
        self.capped = False

    def on_data(self, market_data, balances):

        """Process market data and current balances to make trading decisions.
        
        Args:
            market_data: Dictionary of {pair: tick_data} containing market data for each pair
            balances: Dictionary of {currency: amount} containing current balances
        
        Returns:
            Trading signal dict {pair, side, qty} or None
        """

        orders = []
        if self.capped: return []

        def calculate_portfolio_value():
            """Calculate total portfolio value in fiat currency"""
            value = balances["fiat"]

            # Add token_1 value if we have price data
            if market_data["token_1/fiat"]["close"] is not None:
                value += balances["token_1"] * market_data["token_1/fiat"]["close"]

            # Add token_2 value if we have price data
            if market_data["token_2/fiat"]["close"] is not None:
                value += balances["token_2"] * market_data["token_2/fiat"]["close"]
            # If token_2/fiat price not available but token_1/fiat and token_1/token_2 are available
            elif market_data["token_1/fiat"]["close"] is not None and market_data["token_1/token_2"]["close"] is not None:
                token2_value_in_token1 = balances["token_2"] / market_data["token_1/token_2"]["close"]
                value += token2_value_in_token1 * market_data["token_1/fiat"]["close"]
            return value

        if self.initial_balance == 0: 
            self.initial_balance = calculate_portfolio_value()
            self.prev_balance = calculate_portfolio_value()

        current_balance = calculate_portfolio_value()
        self.profit +=  current_balance - self.prev_balance

        self.prev_balance = current_balance



        # Update price history for each pair
        for pair, data in market_data.items():

            if pair in self.price_history:
                self.price_history[pair].append(data["close"])
                if len(self.price_history[pair]) > self.long_window + 1:
                    self.price_history[pair] = self.price_history[pair][-self.long_window-1:]
                
        
        # Wait until we have enough data points
        for prices in self.price_history.values():
            if len(prices) < self.long_window:
                return orders
        
        # Initialize flag for trading
        if not self.initialized:
            self.initialized = True
            return orders
        
        # Check for trading opportunities in token_1/fiat
        if "token_1/fiat" in market_data:
            prices = self.price_history["token_1/fiat"]
            price = prices[-1]

            short_prices = self.price_history["token_1/fiat"][-self.short_window:]
            long_prices = self.price_history["token_1/fiat"][-self.long_window:]

            momentum = prices[-1] - prices[-self.momentum_window]

            short_ma = np.mean(short_prices)
            long_ma = np.mean(long_prices)

            sigma = np.std(prices)
            
            base_qty = 0.01
            max_fraction = 0.1  # 10% of fiat
            vol_factor = 1 - min(sigma / price, 0.5)
            qty = min(base_qty * vol_factor, balances["fiat"] * max_fraction / price)
            qty = max(0.005, qty)  # never trade less than 0.005
            
            # Calculate RSI

            data = pd.Series(prices)
            delta = data.diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)

            avg_gain = gain.rolling(window=14).mean()
            avg_loss = loss.rolling(window=14).mean()

            rs = avg_gain / avg_loss
            rsivalue = 100 - (100 / (1 + rs))
            rsivalue = rsivalue.iloc[-1]

            booleanMABuy = short_ma >= long_ma and rsivalue < 30
            booleanMASell = short_ma <= long_ma and rsivalue > 70

            if   booleanMABuy and momentum > 0:
                # Get fee from market_data if available, otherwise use default
                fee = market_data["fee"]

                required_fiat = qty * price * (1 + fee)
                if balances["fiat"] <= required_fiat:
                    orders.append({"pair": "token_1/fiat", "side": "buy", "qty": qty})
            
            elif booleanMASell and momentum < 0:

                price_change = data.pct_change()
                drop_amount = price_change.iloc[-1]  # Last price change
                drop_detected = False
                drop_magnitude = 0

                if drop_amount < -0.05:
                    drop_detected = True
                    drop_magnitude = abs(drop_amount)
    
                # If large drop is detected, scale the qty
                if drop_detected:
                    # Scale sell qty: if drop is -5% and threshold is 3%, factor = 5 / 3 = ~1.67 → capped at 1
                    scale_factor = min(drop_magnitude / 0.05, 1.0)
                    scaled_qty = balances["token_2"] * scale_factor
                else:
                    scaled_qty = min(qty, balances["token_2"])
                
                if scaled_qty > 0:
                    orders.append({"pair": "token_2/fiat", "side": "sell", "qty": scaled_qty})
        
        # Check for trading opportunities in token_2/fiat
        if "token_2/fiat" in market_data:
            prices = self.price_history["token_2/fiat"]
            price = prices[-1]

            short_prices = self.price_history["token_2/fiat"][-self.short_window:]
            long_prices = self.price_history["token_2/fiat"][-self.long_window:]

            momentum = prices[-1] - prices[-self.momentum_window]

            short_ma = np.mean(short_prices)
            long_ma = np.mean(long_prices)

            mu, sigma = np.mean(prices), np.std(prices)
            
            base_qty = 0.1
            max_fraction = 0.1  # 10% of fiat
            vol_factor = 1 - min(sigma / price, 0.5)
            qty = min(base_qty * vol_factor, balances["fiat"] * max_fraction / price)
            qty = max(0.05, qty)  # never trade less than 0.005

            data = pd.Series(prices)
            delta = data.diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)

            avg_gain = gain.rolling(window=14).mean()
            avg_loss = loss.rolling(window=14).mean()

            rs = avg_gain / avg_loss
            rsivalue = 100 - (100 / (1 + rs))
            rsivalue = rsivalue.iloc[-1]

            booleanMABuy = short_ma >= long_ma and rsivalue < 30
            booleanMASell = short_ma <= long_ma and rsivalue > 70

            # Buy token_2 with fiat if we have enough fiata
            if booleanMABuy and momentum > 0:
                # Get fee from market_data if available, otherwise use default
                fee = market_data["fee"]

                required_fiat = qty * price * (1 + fee)
                if balances["fiat"] >= required_fiat:
                    orders.append({"pair": "token_2/fiat", "side": "buy", "qty": qty})
            
            elif booleanMASell and momentum < 0:
                price_change = data.pct_change()
                drop_amount = price_change.iloc[-1]  # Last price change
                drop_detected = False
                drop_magnitude = 0
                if drop_amount < -0.05:
                    drop_detected = True
                    drop_magnitude = abs(drop_amount)

                # If large drop is detected, scale the qty
                if drop_detected:
                    # Scale sell qty: if drop is -5% and threshold is 3%, factor = 5 / 3 = ~1.67 → capped at 1
                    scale_factor = min(drop_magnitude / 0.05, 1.0)
                    scaled_qty = balances["token_2"] * scale_factor
                else:
                    scaled_qty = min(qty, balances["token_2"])
                
                if scaled_qty > 0:
                    orders.append({"pair": "token_2/fiat", "side": "sell", "qty": scaled_qty})
        


        # Check for arbitrage opportunities with token_1/token_2
        if all(pair in market_data for pair in ["token_1/fiat", "token_2/fiat", "token_1/token_2"]):
            token1_price = market_data["token_1/fiat"]["close"]
            token2_price = market_data["token_2/fiat"]["close"]
            token1_token2_price = market_data["token_1/token_2"]["close"]
            
            # Calculate implied token_1/token_2 price
            implied_token1_token2 = token1_price / token2_price
            
            # If actual token_1/token_2 price is significantly lower than implied
            if token1_token2_price < implied_token1_token2 * 0.995:
                # Buy token_1 with token_2 (if we have token_2)
                qty_token1 = 0.01
                # Get fee from market_data if available, otherwise use default
                fee = market_data["fee"]
                required_token2 = qty_token1 * token1_token2_price * (1 + fee)
                if balances["token_2"] >= required_token2:
                    orders.append({"pair": "token_1/token_2", "side": "buy", "qty": qty_token1})
            
            # If actual token_1/token_2 price is significantly higher than implied
            elif token1_token2_price > implied_token1_token2 * 1.005:
                # Sell token_1 for token_2 (if we have token_1)
                qty_token1 = min(0.01, balances["token_1"])  # Adjust qty based on available balance
                if qty_token1 > 0:
                    orders.append({"pair": "token_1/token_2", "side": "sell", "qty": qty_token1})

            

            if self.profit/self.initial_balance >= 0.075:
                orders = []
                orders.append({"pair": "token_1/fiat", "side": "sell", "qty": balances["token_1"]})
                orders.append({"pair": "token_2/fiat", "side": "sell", "qty": balances["token_2"]})
                self.capped = True

            print(f"Balance: {balances["fiat"] + balances["token_1"]*self.price_history["token_1/fiat"][-1] + balances["token_2"]*self.price_history["token_2/fiat"][-1]}")
            return orders

strategy = DefaultStrategy()

## Submission File Generation

PARTICIPANTS SHOULD NOT MODIFY THIS FUNCTION.
It is part of the evaluation process to ensure the strategy works as intended.
This function is used to validate the strategy. It runs a backtest on the strategy's performance using historical market data.

In [88]:
class Trader:
    """Trader supporting multiple trading pairs and currencies."""

    def __init__(self, balances, fee):
        # Initialize balances for each currency
        self.balances = balances

        # Track market prices for each pair
        self.prices = {
            "token_1/fiat": None,
            "token_2/fiat": None,
            "token_1/token_2": None
        }

        # First and last prices for reporting
        self.first_prices = {
            "token_1/fiat": None,
            "token_2/fiat": None,
            "token_1/token_2": None
        }

        # Store the first update timestamp for each pair
        self.first_update = {
            "token_1/fiat": False,
            "token_2/fiat": False,
            "token_1/token_2": False
        }

        # Track portfolio value history
        self.equity_history = 0.0
        self.turnover = 0.0
        self.trade_count = 0
        self.total_fees_paid = 0.0  # Track total fees paid

        # Trading fee
        self.fee = fee

    def update_market(self, pair, price_data):
        """Update market prices for a specific trading pair"""
        # Store the updated price
        self.prices[pair] = price_data["close"]

        # Store first price for each pair (for reporting)
        if not self.first_update[pair]:
            self.first_prices[pair] = price_data["close"]
            self.first_update[pair] = True

        # Calculate total portfolio value (in fiat)
        equity = self.calculate_portfolio_value()
        self.equity_history.append(equity)

    def calculate_portfolio_value(self):
        """Calculate total portfolio value in fiat currency"""
        value = self.balances["fiat"]

        # Add token_1 value if we have price data
        if self.prices["token_1/fiat"] is not None:
            value += self.balances["token_1"] * self.prices["token_1/fiat"]

        # Add token_2 value if we have price data
        if self.prices["token_2/fiat"] is not None:
            value += self.balances["token_2"] * self.prices["token_2/fiat"]
        # If token_2/fiat price not available but token_1/fiat and token_1/token_2 are available
        elif self.prices["token_1/fiat"] is not None and self.prices["token_1/token_2"] is not None:
            token2_value_in_token1 = self.balances["token_2"] / self.prices["token_1/token_2"]
            value += token2_value_in_token1 * self.prices["token_1/fiat"]

        return value

    def execute(self, order):
        """Execute a trading order across any supported pair"""
        pair = order["pair"]  # e.g., "token_1/fiat"
        side = order["side"]  # "buy" or "sell"
        qty = float(order["qty"])

        # Split the pair into base and quote currencies
        base, quote = pair.split("/")

        # Get current price for the pair
        price = self.prices[pair]
        if price is None:
            return  # Can't trade without a price

        executed = False

        if side == "buy":
            # Calculate total cost including fee
            base_cost = qty * price
            fee_amount = base_cost * self.fee
            total_cost = base_cost + fee_amount

            # Check if we have enough of the quote currency
            if self.balances[quote] >= total_cost:
                # Deduct quote currency (e.g., fiat)
                self.balances[quote] -= total_cost

                # Add base currency (e.g., token_1)
                self.balances[base] += qty

                # Track turnover and fees
                self.turnover += total_cost
                self.total_fees_paid += fee_amount
                executed = True

        elif side == "sell":
            # Check if we have enough of the base currency
            if self.balances[base] >= qty:
                # Calculate proceeds after fee
                base_proceeds = qty * price
                fee_amount = base_proceeds * self.fee
                net_proceeds = base_proceeds - fee_amount

                # Add quote currency (e.g., fiat)
                self.balances[quote] += net_proceeds

                # Deduct base currency (e.g., token_1)
                self.balances[base] -= qty

                # Track turnover and fees
                self.turnover += base_proceeds
                self.total_fees_paid += fee_amount
                executed = True

        # Count successful trades
        if executed:
            self.trade_count += 1

In [89]:
def run_backtest(combined_data: pd.DataFrame, fee: float, balances: dict[str, float]) -> pd.DataFrame:
    """Run a backtest with multiple trading pairs.

    Args:
        submission_dir: Path to the strategy directory
        combined_data: DataFrame containing market data for multiple pairs
        fee: Trading fee (in basis points, e.g., 2 = 0.02%)
        balances: Dictionary of {pair: amount} containing initial balances
    """
    # Record initial balances for display
    trader = Trader(balances, fee)

    initial_balances = balances.copy()

    # Initialize prices with first data point for each pair
    combined_data.sort_values("timestamp", inplace=True)
    first_prices = {k: df.iloc[0]['close'] for k, df in combined_data.groupby("symbol")}

    # Calculate true initial portfolio value including all assets
    initial_portfolio_value = initial_balances["fiat"]
    if "token_1/fiat" in first_prices and initial_balances["token_1"] > 0:
        initial_portfolio_value += initial_balances["token_1"] * first_prices["token_1/fiat"]
    if "token_2/fiat" in first_prices and initial_balances["token_2"] > 0:
        initial_portfolio_value += initial_balances["token_2"] * first_prices["token_2/fiat"]

    trader.equity_history = [initial_portfolio_value]
    # Combine all dataframes and sort by timestamp
    result = pd.DataFrame(
        columns=["id", "timestamp", "pair", "side", "qty"],
    )

    # Process data timestamp by timestamp
    for timestamp, group in combined_data.groupby('timestamp'):
        # Update prices for each pair in this timestamp
        market_data = {
            "fee": fee,
        }
        for _, row in group.iterrows():
            pair = row['symbol']
            data_dict = row.to_dict()
            # Add fee information to market data so strategies can access it
            market_data[pair] = data_dict
            trader.update_market(pair, data_dict)
        
        # Get strategy decision based on all available market data and current balances
        orders = strategy.on_data(market_data, balances)

        # Handle list of orders
        for order in orders:
            trader.execute(order)
            order["timestamp"] = timestamp
            order["id"] = str(uuid.uuid4())
            result = pd.concat([result, pd.DataFrame([order])], ignore_index=True)

    return result

In [90]:
with open("./input/hyperparameters.json") as f:
    HYPERPARAMETERS = json.load(f)
    
FEE = HYPERPARAMETERS.get("fee", 3.0)/10000.0  # Convert to decimal
BALANCE_FIAT = HYPERPARAMETERS.get("fiat_balance", 10000.0)
BALANCE_TOKEN1 = HYPERPARAMETERS.get("token1_balance", 0.0)
BALANCE_TOKEN2 = HYPERPARAMETERS.get("token2_balance", 0.0)
OUTPUT = "submission.csv"

combined_data = pd.read_csv("./input/test.csv")

# Run the backtest on the provided test data with a fee of 0.02% and initial balances of 10,000 fiat, and 0 token_1 and token_2
result = run_backtest(combined_data, FEE, {
    "fiat": BALANCE_FIAT,
    "token_1": BALANCE_TOKEN1,
    "token_2": BALANCE_TOKEN2,
})

# Output the backtest result to a CSV file for submission
result.to_csv(OUTPUT, index=False)

Balance: 1817010.0
Balance: 1817476.7000000002
Balance: 1817333.5
Balance: 1818472.0
Balance: 1817776.2000000002
Balance: 1815280.0
Balance: 1813926.7
Balance: 1813645.2
Balance: 1813027.6
Balance: 1813882.8
Balance: 1814478.9
Balance: 1814340.9
Balance: 1815420.5
Balance: 1816464.0
Balance: 1816605.7
Balance: 1817494.7999999998
Balance: 1817062.4
Balance: 1818235.2999999998
Balance: 1819488.0
Balance: 1817505.7
Balance: 1817674.2999999998
Balance: 1818837.2
Balance: 1819152.4
Balance: 1818049.3
Balance: 1814813.2000000002
Balance: 1815100.0
Balance: 1817114.7000000002
Balance: 1816605.4
Balance: 1817698.2000000002
Balance: 1818090.0
Balance: 1817567.5
Balance: 1815987.5
Balance: 1817614.1
Balance: 1819123.3
Balance: 1818780.0
Balance: 1819495.2000000002
Balance: 1820354.6
Balance: 1820750.4
Balance: 1818925.9
Balance: 1818685.2000000002
Balance: 1817914.8
Balance: 1816928.7000000002
Balance: 1816261.4
Balance: 1818484.2000000002
Balance: 1819587.9
Balance: 1820660.7999999998
Balance: 

C:\Users\rafey\AppData\Local\Temp\ipykernel_13348\3259639209.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, pd.DataFrame([order])], ignore_index=True)


Balance: 1834906.7185867294
Balance: 1835493.189182723
Balance: 1835828.8502951139
Balance: 1836630.7201418702
Balance: 1836458.1957737594
Balance: 1836512.4260928566
Balance: 1834775.6873951238
Balance: 1835290.1951859007
Balance: 1835425.249251947
Balance: 1834104.4252395662
Balance: 1832320.5401621812
Balance: 1833529.9795095734
Balance: 1833205.858403843
Balance: 1833905.3130388597
Balance: 1834384.9024550943
Balance: 1835104.990349161
Balance: 1834875.37202681
Balance: 1836685.994246787
Balance: 1834627.1095203273
Balance: 1831217.5282413198
Balance: 1829423.1330591706
Balance: 1830214.0772669958
Balance: 1829530.1622592164
Balance: 1830527.4207540816
Balance: 1833919.1224939646
Balance: 1836649.6815718836
Balance: 1837858.839142049
Balance: 1836941.0522959535
Balance: 1835943.8714719242
Balance: 1836818.6928478905
Balance: 1837479.0877646082
Balance: 1836938.6991449976
Balance: 1838855.7212890899
Balance: 1833332.0714437556
Balance: 1830265.8173969276
Balance: 1831073.5455205042
